# KoBert 기본

In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import BertForSequenceClassification, BertModel
from kobert_tokenizer import KoBERTTokenizer
from tqdm import tqdm

/home/bok/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 모델 및 토큰 불러오기
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
model = BertForSequenceClassification.from_pretrained('skt/kobert-base-v1')

# 입력 문장 정의
input_text = '안녕하세요 반갑습니다'

# 입력 문장 토큰화
inputs = tokenizer(input_text, return_tensors="pt")

# 모델 예측
outputs = model(**inputs)

# 출력 로짓(logits)을 소프트맥스 확률로 변환
probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)

# 가장 높은 확률을 가지는 클래스 선택
predicted_class = torch.argmax(probabilities, dim=-1).item()

# 감정 레이블 정의 (모델에 따라 다를 수 있음)
labels = ["negative", "neutral", "positive"]

# 예측된 감정 출력
print(f"문장: {input_text}")
print(f"예측된 감정: {labels[predicted_class]}")
print(f"확률: {probabilities.tolist()}")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at skt/kobert-base-v1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


문장: 안녕하세요 반갑습니다
예측된 감정: negative
확률: [[0.5940529108047485, 0.40594708919525146]]


# 학습

In [3]:
# 데이터 불러오기
import pandas as pd

df = pd.read_excel('./model/data/train_set.xlsx')
df.head()

,question,label
0,데이터프레임에서 특정 값을 기준으로 데이터를 필터링하려면 어떻게 해야 하나요?,yes
1,데이터프레임의 인덱스를 변경하려면 어떻게 해야 하나요?,yes
2,데이터프레임의 컬럼 이름을 변경하려면 어떻게 해야 하나요?,yes
3,데이터 프레임에 새로운 행을 추가하려면 어떻게 해야 하나요?,yes
4,데이터프레임의 행을 삭제하려면 어떻게 해야 하나요?,yes


In [4]:
# 데이터셋 정의
class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            max_length=self.max_len,
            truncation=True,
            padding='max_length',
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [5]:
# 데이터 준비
texts = df.question.tolist()
df.label = df.label.replace({'yes': 1, 'no': 0})
labels = df.label.tolist()

max_len = 128
batch_size = 2

dataset = SentimentDataset(texts, labels, tokenizer, max_len)
data_loader = DataLoader(dataset, batch_size, shuffle=True)

/tmp/ipykernel_90033/1444296744.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.label = df.label.replace({'yes': 1, 'no': 0})


In [6]:
# 모델 가중치 초기화 함수
def initialize_weights(module):
    if isinstance(module, (nn.Linear, nn.Embedding)):
        module.weight.data.normal_(mean=0.0, std=0.02)
    elif isinstance(module, nn.LayerNorm):
        module.bias.data.zero_()
        module.weight.data.fill_(1.0)
    if isinstance(module, nn.Linear) and module.bias is not None:
        module.bias.data.zero_()

# 모델의 모든 가중치 초기화
model.apply(initialize_weights)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-

In [7]:
# 학습 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

optimizer = optim.AdamW(model.parameters(), lr=5e-5)
loss_fn = nn.CrossEntropyLoss().to(device)

In [8]:
# 학습 파라미터 정의
def train_epoch(model, data_loader, loss_fn, optimizer, device, scheduler, n_examples):
    model = model.train()
    losses = 0
    correct_predictions = 0

    for batch in tqdm(data_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )

        loss = outputs.loss
        logits = outputs.logits

        _, preds = torch.max(logits, dim=1)
        correct_predictions += torch.sum(preds == labels)
        losses += loss.item()

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    return correct_predictions.double() / n_examples, losses / n_examples


In [9]:
# 학습
epochs = 3
for epoch in range(epochs):
    print(f'Epoch {epoch + 1}/{epochs}')
    print('-' * 10)
    train_acc, train_loss = train_epoch(
        model,
        data_loader,
        loss_fn,
        optimizer,
        device,
        None,
        len(dataset)
    )
    print(f'Train loss {train_loss} accuracy {train_acc}')


Epoch 1/3
----------


100%|██████████| 179/179 [00:33<00:00,  5.41it/s]


Train loss 0.11701876103881924 accuracy 0.888268156424581
Epoch 2/3
----------


100%|██████████| 179/179 [00:32<00:00,  5.46it/s]


Train loss 0.4098627818839406 accuracy 0.4776536312849162
Epoch 3/3
----------


100%|██████████| 179/179 [00:33<00:00,  5.40it/s]


Train loss 0.3524361500443693 accuracy 0.5363128491620112


In [19]:
# 모델 저장 경로 설정
def get_model_save_path(base_path, base_filename, ext):
    version = 1
    while True:
        model_save_path = f"{base_path}/{base_filename}_v{version}{ext}"
        if not os.path.exists(model_save_path):
            return model_save_path
        version += 1

# 모델 저장
base_path = './model/save'
base_filename = 'model'
ext = '.pth'

model_save_path = get_model_save_path(base_path, base_filename, ext)
torch.save(model.state_dict(), model_save_path)
print(f'Model saved to {model_save_path}')

Model saved to ./model/save/model_v4.pth


# 검증

In [11]:
validation_df = pd.read_excel('./model/data/valid_set.xlsx')
validation_df.head()

,question
0,파이썬에서 리스트와 튜플의 차이는 무엇인가요?
1,파이썬에서 딕셔너리를 어떻게 정의하나요?
2,파이썬에서 클래스는 어떻게 생성하나요?
3,파이썬에서 파일을 읽고 쓰는 방법은 무엇인가요?
4,파이썬의 주요 내장 함수들은 무엇이 있나요?


In [12]:
# 검증할 데이터 정의
class QuestionDataset(Dataset):
    def __init__(self, texts, tokenizer, max_len):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        encoding = self.tokenizer(
            text,
            max_length=self.max_len,
            truncation=True,
            padding='max_length',
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
        }

In [13]:
# 모델 불러오기
model = BertForSequenceClassification.from_pretrained('skt/kobert-base-v1', num_labels=2)  # 모델의 아키텍처 설정
model_save_path = './model/save/model_v1.pth'  # 가중치 불러오기
model.load_state_dict(torch.load(model_save_path))
model = model.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at skt/kobert-base-v1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
# 검증할 데이터 준비
validation_texts = validation_df['question'].tolist()

max_len = 128
batch_size = 2

tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
validation_dataset = QuestionDataset(validation_texts, tokenizer, max_len)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [15]:
# 예측 함수 정의
def predict(model, data_loader, device):
    model = model.eval()
    predictions = []

    with torch.no_grad():
        for batch in tqdm(data_loader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )

            logits = outputs.logits
            _, preds = torch.max(logits, dim=1)
            predictions.extend(preds.cpu().numpy())

    return predictions

In [16]:
# 예측
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 모델 로드 및 가중치 초기화 (이미 초기화된 모델 사용)
model = BertForSequenceClassification.from_pretrained('skt/kobert-base-v1', num_labels=2)
model.apply(initialize_weights)
model = model.to(device)

# 예측 수행
predictions = predict(model, validation_loader, device)

# 결과 출력
for text, pred in zip(validation_texts, predictions):
    label = "yes" if pred == 1 else "no"
    print(f"문장: {text} -> 예측된 레이블: {label}")
    

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at skt/kobert-base-v1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 75/75 [00:02<00:00, 25.01it/s]

문장: 파이썬에서 리스트와 튜플의 차이는 무엇인가요? -> 예측된 레이블: yes
문장: 파이썬에서 딕셔너리를 어떻게 정의하나요? -> 예측된 레이블: yes
문장: 파이썬에서 클래스는 어떻게 생성하나요? -> 예측된 레이블: yes
문장: 파이썬에서 파일을 읽고 쓰는 방법은 무엇인가요? -> 예측된 레이블: yes
문장: 파이썬의 주요 내장 함수들은 무엇이 있나요? -> 예측된 레이블: yes
문장: 파이썬에서 for 루프의 기본 구조는 어떻게 되나요? -> 예측된 레이블: yes
문장: 파이썬에서 예외 처리는 어떻게 하나요? -> 예측된 레이블: yes
문장: 파이썬에서 패키지를 설치하는 명령어는 무엇인가요? -> 예측된 레이블: yes
문장: 파이썬의 주요 데이터 타입에는 무엇이 있나요? -> 예측된 레이블: yes
문장: 파이썬에서 함수는 어떻게 정의하나요? -> 예측된 레이블: yes
문장: 파이썬에서 리스트의 요소를 정렬하는 방법은 무엇인가요? -> 예측된 레이블: yes
문장: 파이썬에서 문자열 포매팅은 어떻게 하나요? -> 예측된 레이블: yes
문장: 파이썬에서 lambda 함수는 무엇인가요? -> 예측된 레이블: yes
문장: 파이썬에서 모듈과 패키지의 차이는 무엇인가요? -> 예측된 레이블: yes
문장: 파이썬에서 글로벌 변수와 로컬 변수의 차이는 무엇인가요? -> 예측된 레이블: yes
문장: 파이썬의 주요 표준 라이브러리에는 무엇이 있나요? -> 예측된 레이블: yes
문장: 파이썬에서 set 자료형은 어떻게 사용하나요? -> 예측된 레이블: yes
문장: 파이썬에서 map 함수는 어떻게 사용하나요? -> 예측된 레이블: yes
문장: 파이썬에서 데코레이터는 무엇인가요? -> 예측된 레이블: yes
문장: 파이썬에서 제너레이터는 무엇인가요? -> 예측된 레이블: yes
문장: 파이썬에서 리스트 컴프리헨션은 어떻게 사용하나요? -> 예측된 레이블: yes
문장: 파이썬에서 가비지 컬렉션은 어떻게 동작하나요? -> 예측된 레이블: yes
